## Homework

В этой домашней работе вы будете предсказывать стоимость домов по их характеристикам.

Метрика качества: `RMSE`

Оценивание:
* Baseline - 2 балла
* Feature Engineering - 2 балла
* Model Selection - 3 балла
* Ensemble v.1 - 3 балла
* (*) Ensemble v.2 - дополнительно, 2 балла

### Описание датасета

Короткое описание данных:
```
price: sale price (this is the target variable)
id: transaction id
timestamp: date of transaction
full_sq: total area in square meters, including loggias, balconies and other non-residential areas
life_sq: living area in square meters, excluding loggias, balconies and other non-residential areas
floor: for apartments, floor of the building
max_floor: number of floors in the building
material: wall material
build_year: year built
num_room: number of living rooms
kitch_sq: kitchen area
state: apartment condition
product_type: owner-occupier purchase or investment
sub_area: name of the district

The dataset also includes a collection of features about each property's surrounding neighbourhood, and some features that are constant across each sub area (known as a Raion). Most of the feature names are self explanatory, with the following notes. See below for a complete list.

full_all: subarea population
male_f, female_f: subarea population by gender
young_*: population younger than working age
work_*: working-age population
ekder_*: retirement-age population
n_m_{all|male|female}: population between n and m years old
build_count_*: buildings in the subarea by construction type or year
x_count_500: the number of x within 500m of the property
x_part_500: the share of x within 500m of the property
_sqm_: square meters
cafe_count_d_price_p: number of cafes within d meters of the property that have an average bill under p RUB
trc_: shopping malls
prom_: industrial zones
green_: green zones
metro_: subway
_avto_: distances by car
mkad_: Moscow Circle Auto Road
ttk_: Third Transport Ring
sadovoe_: Garden Ring
bulvar_ring_: Boulevard Ring
kremlin_: City center
zd_vokzaly_: Train station
oil_chemistry_: Dirty industry
ts_: Power plant
```

### Setup

In [435]:
import pandas as pd
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [436]:
df = pd.read_csv("houses_data.csv", parse_dates=["timestamp"])
useful_columns = ['life_sq', 'floor', 'max_floor', 'material', 'build_year', 'num_room', 'kitch_sq',
                  'state', 'metro_km_walk', 'ecology']
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price
0,0,2014-12-26,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,36,7,2,15,33,1,12,75,10,15318960
1,1,2012-10-04,64,64.0,16.0,NaN,NaN,NaN,NaN,NaN,...,2,2,0,0,13,1,0,6,1,6080000
2,2,2014-02-05,83,44.0,9.0,17.0,1.0,1985.0,3.0,10.0,...,13,6,1,8,18,0,1,52,0,17000000
3,3,2012-07-26,71,49.0,2.0,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,3,0,2,8,2,990000
4,4,2014-10-29,60,42.0,9.0,9.0,1.0,1970.0,3.0,6.0,...,3,1,0,5,8,0,1,34,5,7900000


Разделите имеющиеся у вас данные на обучающую и тестовую выборки. В качестве обучающей выборки возьмите первые 80% данных, последние 20% - тестовая выборка.

In [437]:
X = df.drop(['price'], axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Возможно в ваших моделях вам придется указывать, какие колонки являются категориальными (например, в бустингах). Для упрощения предлагается разделить колонки.

In [438]:
drop_columns = [
    'id',           # May leak information
    'timestamp',    # May leak information
]
cat_columns = [
    'product_type',              #
    'material',                  # Material of the wall
    'state',                     # Satisfaction level
    'sub_area',                  # District name
    'culture_objects_top_25',    #
    'thermal_power_plant_raion', #
    'incineration_raion',        #
    'oil_chemistry_raion',       #
    'radiation_raion',           #
    'railroad_terminal_raion',   #
    'big_market_raion',          #
    'nuclear_reactor_raion',     #
    'detention_facility_raion',  #
    'ID_metro',                  #
    'ID_railroad_station_walk',  #
    'ID_railroad_station_avto',  #
    'water_1line',               #
    'ID_big_road1',              #
    'big_road1_1line',           #
    'ID_big_road2',              #
    'railroad_1line',            #
    'ID_railroad_terminal',      #
    'ID_bus_terminal',           #
    'ecology',                   #
]

num_columns = list(set(X_train.columns).difference(set(cat_columns + drop_columns)))
# Возьмем все числовые колонки без Nan и некоторые полезные колонки
not_na_num_columns = list(X_train[num_columns].isna().sum()[X_train.isna().sum() == 0].index)
X_train = X_train[not_na_num_columns + useful_columns]
X_test = X_test[not_na_num_columns + useful_columns]

In [439]:
X_train[useful_columns].isna().sum()

life_sq          3267
floor              96
max_floor        5013
material         5013
build_year       7110
num_room         5013
kitch_sq         5013
state            7143
metro_km_walk      12
ecology             0
dtype: int64

**Cначала разберемся с категориальным ecology**

In [440]:
X_train['ecology'].value_counts()

poor            4198
no data         3964
good            3771
excellent       2105
satisfactory    1962
Name: ecology, dtype: int64

In [441]:
def deal_with_ecology(X_train, X_test):
 
    def ecol_miss(value):
        if value == 'no data':
            return 1
        return 0

    X_train['ecology_missing'] = 0
    X_test['ecology_missing'] = 0
    X_train['ecology_missing'] = X_train.apply(lambda row: ecol_miss(row['ecology']), axis=1)
    X_test['ecology_missing'] = X_test.apply(lambda row: ecol_miss(row['ecology']), axis=1)

    score_to_num = {'poor': 0, 'satisfactory': 1, 'good': 2, 'excellent': 3, 'no data': 1}

    X_train['ecology'] = X_train['ecology'].map(score_to_num)
    X_test['ecology'] = X_test['ecology'].map(score_to_num)

**Теперь отдельно с life_sq**

In [442]:
def deal_with_life_sq(X_train, X_test):

    def life_sq_miss(value, full_sq):
        if pd.isna(value):
            return 0.65 * full_sq  # примерно столько в среднем жилая площадь составляет от общей
        return value

    X_train['life_sq'] = X_train.apply(lambda row: life_sq_miss(row['life_sq'], row['full_sq']), axis=1)
    X_test['life_sq'] = X_test.apply(lambda row: life_sq_miss(row['life_sq'], row['full_sq']), axis=1)

**Теперь с остальными**

In [443]:
def deal_with_others(X_train, X_test):
    
    def only_median(X_train, X_test, col):
        median = X_train[col].median()
        X_train[col].fillna(median, inplace=True)
        X_test[col].fillna(median, inplace=True)

    def miss_value(value):
        if pd.isna(value):
            return 1
        return 0

    def median_and_column(X_train, X_test, col):
        X_train[col +'_missing'] = 0
        X_test[col +'_missing'] = 0
        X_train[col +'_missing'] = X_train.apply(lambda row: miss_value(row[col]), axis=1)
        X_test[col +'_missing'] = X_test.apply(lambda row: miss_value(row[col]), axis=1)
        only_median(X_train, X_test, col)


    add_median_and_forget_about_it = ['metro_km_walk', 'floor', 'max_floor', 'num_room']
    add_median_and_column = ['material', 'build_year', 'kitch_sq', 'state']

    for col in add_median_and_forget_about_it:
        only_median(X_train, X_test, col)

    for col in add_median_and_column:
        median_and_column(X_train, X_test, col)

In [444]:
def repair_features(X_train, X_test):
    deal_with_ecology(X_train, X_test)
    deal_with_life_sq(X_train, X_test)
    deal_with_others(X_train, X_test)
    
repair_features(X_train, X_test)
X_train[useful_columns].isna().sum()

life_sq          0
floor            0
max_floor        0
material         0
build_year       0
num_room         0
kitch_sq         0
state            0
metro_km_walk    0
ecology          0
dtype: int64

### Baseline (2 балла)

В качестве Baseline обучите `DecisionTreeRegressor` из `sklearn`.

In [445]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

single_tree = DecisionTreeRegressor()
single_tree.fit(X_train, y_train)

DecisionTreeRegressor()

Проверьте качество на отложенной выборке.

In [446]:
y_pred = single_tree.predict(X_test)
print(f'RMSE: {metrics.mean_squared_error(y_test, y_pred, squared=False):.2f}')

RMSE: 3858855.87


### Feature Engineering (2 балла)

Часто улучшить модель можно с помощью аккуратного Feature Engineering.

Добавим в модель дополнительные признаки:
* "Как часто в этот год и этот месяц появлились объявления"
* "Как часто в этот год и эту неделю появлялись объявления"

In [447]:
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df["month_year_cnt"] = month_year.map(month_year_cnt_map)

week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df["week_year_cnt"] = week_year.map(week_year_cnt_map)

Добавьте следюущие дополнительные признаки:
* Месяц (из колонки `timestamp`)
* День недели (из колонки `timestamp`)
* Отношение "этаж / максимальный этаж в здании" (колонки `floor` и `max_floor`)
* Отношение "площадь кухни / площадь квартиры" (колонки `kitchen_sq` и `full_sq`)

По желанию можно добавить и другие признаки.

In [448]:
df['month'] = df['timestamp'].apply(lambda time: time.month)
df['day_of_week'] = df['timestamp'].apply(lambda time: time.day_of_week)
df['floor_to_max'] = df['floor'] / df['max_floor']
df['kitchen_size'] = df['kitch_sq'] / df['full_sq']

Разделите выборку на обучающую и тестовую еще раз (потому что дополнительные признаки созданы для исходной выборки).

In [449]:
X = df.drop(['price'], axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train[not_na_num_columns + useful_columns]
X_test = X_test[not_na_num_columns + useful_columns]
repair_features(X_train, X_test)

### Model Selection (3 балла)

Посмотрите, какого качества можно добиться если использовать разные модели:
* `DecisionTreeRegressor` из `sklearn`
* `RandomForestRegressor` из `sklearn`
* `CatBoostRegressor`

Также вы можете попробовать линейные модели, другие бустинги (`LigthGBM` и `XGBoost`).

Почти все библиотеки поддерживают удобный способ подбора гиперпараметров: посмотрите как это делать в [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) или в [catboost](https://catboost.ai/docs/concepts/python-reference_catboostregressor_grid_search.html).

Проверяйте качество каждой модели на тестовой выборке и выберите наилучшую.

In [465]:
single_tree = DecisionTreeRegressor()
single_tree.fit(X_train, y_train)

y_pred = single_tree.predict(X_test)
print(f'RMSE на одном дереве: {metrics.mean_squared_error(y_test, y_pred, squared=False):.2f}')

RMSE на одном дереве: 3906593.91


In [453]:
single_forest = RandomForestRegressor()
single_forest.fit(X_train, y_train)

y_pred = single_forest.predict(X_test)
print(f'RMSE на лесе деревьев: {metrics.mean_squared_error(y_test, y_pred, squared=False):.2f}')

RMSE на лесе деревьев: 2659761.73


In [464]:
from catboost import CatBoostRegressor
model = CatBoostRegressor(iterations=300,
                          learning_rate=0.2,
                          depth=2)

model.fit(X_train, y_train, verbose=0)
y_pred = model.predict(X_test)
print(f'RMSE на Catboost: {metrics.mean_squared_error(y_test, y_pred, squared=False):.2f}')

RMSE на Catboost: 2673186.02


**Давайте попробуем grid_search c двумя самыми многообещающими моделями**

In [470]:
from sklearn.model_selection import GridSearchCV  # Лес сильно мучать не будем
parameters = {'n_estimators': [200], 'min_samples_leaf':[3, 6], 'max_depth':[3, 5], 
              'n_jobs' : [-1], 'max_features': ['auto', 'sqrt']}
single_forest = RandomForestRegressor()
model = GridSearchCV(single_forest, parameters, verbose=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'RMSE на лесу с GridSearch: {metrics.mean_squared_error(y_test, y_pred, squared=False):.2f}')
# не получилось, не фортануло, ну и ладно, у нас есть Catboost

Fitting 5 folds for each of 8 candidates, totalling 40 fits
RMSE на лесу с GridSearch: 2846702.65


In [ ]:
model = CatBoostRegressor()  # Да и Catboost до дедлайна не созреет...

grid = {'iterations': [1000],
        'learning_rate': [0.03, 0.1, 0.3],
        'depth': [3, 6],
        'l2_leaf_reg': [1, 3]}

grid_search_result = model.grid_search(grid,
                                       X=X_train,
                                       y=y_train,
                                       plot=True)

In [474]:
model = CatBoostRegressor(iterations=1000,
                          learning_rate=0.1,
                          depth=6,
                          l2_leaf_reg = 3)  # параметры по результат GridSearch

model.fit(X_train, y_train, verbose=0)
y_pred = model.predict(X_test)
print(f'RMSE на Catboost с GridSearch: {metrics.mean_squared_error(y_test, y_pred, squared=False):.2f}')

RMSE на Catboost с GridSearch: 2555042.26


### Ensemble v.1 (3 балла)

Ансамбли иногда оказываются лучше чем одна большая модель.

В колонке `product_type` содержится информация о том, каким является объявление: `Investment` (продажа квартиры как инвестиции) или `OwnerOccupier` (продажа квартиры для жилья). Логично предположить, что если сделать по модели на каждый из этих типов, то качество будет выше.

Обучите свои лучшие модели на отдельно на `Investment` и `OwnerOccupier` (т.е. у вас будет `model_invest`, обученная на `(invest_train_X, invest_train_Y)` и `model_owner`, обученная на `(owner_train_X, owner_train_Y)`) и проверьте качество на отложенной выборке (т.е. на исходном `test_split`).

In [478]:
df_investment = df[df['product_type'] == 'Investment']
df_owner = df[df['product_type'] == 'OwnerOccupier']

X_investment = df_investment.drop(['price'], axis=1)
y_investment = df_investment['price']

X_owner = df_owner.drop(['price'], axis=1)
y_owner = df_owner['price']

X_train_investment, X_test_investment, y_train_investment, y_test_investment = train_test_split(
    X_investment, y_investment, test_size=0.2, random_state=42)
    
X_train_owner, X_test_owner, y_train_owner, y_test_owner = train_test_split(
    X_owner, y_owner, test_size=0.2, random_state=42)

In [479]:
X_train_investment = X_train_investment[not_na_num_columns + useful_columns]
X_test_investment = X_test_investment[not_na_num_columns + useful_columns]

X_train_owner = X_train_owner[not_na_num_columns + useful_columns]
X_test_owner = X_test_owner[not_na_num_columns + useful_columns]

repair_features(X_train_investment, X_test_investment)
repair_features(X_train_owner, X_test_owner)

In [480]:
model_investment = CatBoostRegressor(iterations=1000,
                          learning_rate=0.1,
                          depth=6,
                          l2_leaf_reg = 3)
model_owner = CatBoostRegressor(iterations=1000,
                          learning_rate=0.1,
                          depth=6,
                          l2_leaf_reg = 3)

model_investment.fit(X_train_investment, y_train_investment, verbose=0)
model_owner.fit(X_train_owner, y_train_owner, verbose=0)
y_pred_investment = model_investment.predict(X_test_investment)
y_pred_owner = model_owner.predict(X_test_owner)

In [498]:
y_pred = np.hstack((y_pred_investment, y_pred_owner))
y_test = np.hstack((y_test_investment, y_test_owner))

print(f'RMSE на Catboost с разделением по product_type: {metrics.mean_squared_error(y_test, y_pred, squared=False):.2f}')

RMSE на Catboost с разделением по product_type: 2471087.69


**Реально лучше!**

### (*) Ensemble v.2 (дополнительно, 2 балла)

Попробуйте сделать для `Investment` более сложную модель: обучите `CatBoostRegressor` и `HuberRegressor` из `sklearn`, а затем сложите их предсказания с весами `w_1` и `w_2` (выберите веса сами; сумма весов равняется 1).

In [503]:
print(f'Начальное RMSE на Investment: {metrics.mean_squared_error(y_test_investment, y_pred_investment, squared=False):.2f}')

Начальное RMSE на Investment: 2978134.14


In [504]:
from sklearn.linear_model import HuberRegressor
huber = HuberRegressor().fit(X_train_investment, y_train_investment)

In [505]:
y_pred_investment_huber = huber.predict(X_test_investment)

In [513]:
best_w = 0
best_RMSE = 1e10
for w in np.arange(0, 1, 0.01):
    y_pred_investment_ensemble = w * y_pred_investment_huber + (1 - w) * y_pred_investment
    RMSE = metrics.mean_squared_error(y_test_investment, y_pred_investment_ensemble, squared=False)
    if RMSE < best_RMSE:
        best_RMSE = RMSE
        best_w = w

In [518]:
print(f'Получившееся RMSE на Investment: {best_RMSE:.2f}')
print(f'Вес при Huber: {best_w}')

Получившееся RMSE на Investment: 2976295.29
Вес при Huber: 0.03


**Huber действительно помог улучшить результат, хоть и немного, но и много времени на его обучение не тратилось**